In [1]:
#! /home/croth/anaconda2/bin/python
## At what temperatrue and amph B. concentration?
T = 30
D = 0.000
## set permutations
perms = 2

In [ ]:
## Set date of phenotype data to use
filedate = 'Oct152018'

In [2]:
## set path to phenotype data, should in in the words, "curation_baselined_median21_AUC_Xstrain.csv'
file_path = '../FILES/Tecan_qtlruns_%s_curated_baselined_median21_AUC_Xstrain.csv'%(filedate)
## Set file paths
genopath = '../FILES/CDx-ill-SNP-df-104-blocked.csv'
## Set save path
savepathdir = '../FILES/PERMUT/'

In [3]:
## Set the parameter of interest
param = 'median_AUC'

In [4]:
## Import mods 
import numpy as np, pandas as pd, os

In [5]:
## Bring in needed ftn for QTL analysis
from scipy.stats import kruskal

In [6]:
## Write function for QTL analysis
def crypto_kruskal(site,pheno):
    """
    For the data from the C. deneoformans cross, a non-parametric ANOVA.
    Here we are looking for differences between the median phenotypes of group 0 and 1. 
    """
    pheno = pheno.dropna();assert len(pheno) > 0;
    site = site[pheno.dropna().index.tolist()];
    assert len(site) == len(pheno)
    try:
        pval = -np.log10(kruskal(np.array(pheno)[np.array(site) == 0],
                                        np.array(pheno)[np.array(site) == 1],
                                        nan_policy='omit')[1])
    except ValueError:
        pval = np.nan
    return pval

In [7]:
## Set seed number 
seed_num = 71191

In [8]:
## Seed random numbers genorator
np.random.seed(seed_num)

In [9]:
## Check if path exitsts and make it if not. 
if not os.path.exists(savepathdir):
    os.mkdir(savepathdir)

In [10]:
## Bring in genotype file 
gdfs = pd.read_csv(genopath,index_col=0)

In [11]:
## set the column with samples in it. 
tc = gdfs.columns.tolist().index('Chrom') + 1

In [12]:
## Bring in phenotype file
phenos = pd.read_csv(file_path,index_col=0)

In [13]:
## Set pheno index
phenos.index = phenos['strain']

In [14]:
## set info columns
info_colu = gdfs.T.iloc[:tc].T.columns.tolist()

In [15]:
## Check work
assert len(info_colu) == tc

In [16]:
## gather samples
samples = gdfs.T.iloc[tc:].T.columns.tolist()

In [17]:
## Rename samples to match our phenotype data
bgeno_strains = []
for s in samples:
    if len(s.split('_Cor')) > 1:
        bgeno_strains.append(s.split('_Cor')[0]+'_cor')
    elif s in ['SS-A853']:
        bgeno_strains.append('XL280alpha')
    elif s in ['SS-B830']:
        bgeno_strains.append('XL280a')
    else:
        bgeno_strains.append(s)

In [18]:
## Check our work
assert len(samples) == len(bgeno_strains)

In [19]:
## rename our samples
gdfs.columns = info_colu + bgeno_strains

In [20]:
## Take the param of interest
pheno = phenos[(phenos.temp==T) & (phenos.amphB==D)].loc[bgeno_strains][param]

In [21]:
## Set strain index
strain_ix = pheno.index

In [22]:
## Conducted permuted qtl analysis
perms_dfs = []
pheno_dfs = []
for ph in range(perms):
    ## Shuffle the phenotype data
    df = pheno.sample(frac=1).reset_index(drop=True)
    df.index = strain_ix
    ## Conduct our qtl mapping with the non parametric ANOVA
    perm_pval = gdfs.iloc[:,tc:].apply(crypto_kruskal,
                                       axis=1,
                                       args=[df])
    ## Save the data
    perms_dfs.append(perm_pval)
    pheno_dfs.append(df)

In [23]:
## concat pvalues dfs
permdf = pd.concat([gdfs[info_colu]]+perms_dfs,axis=1)

In [24]:
## Set new column names
permdf.columns = info_colu + [param+'_%sC_%s_p%s'%(T,D,i) for i in range(perms)]

In [25]:
## save permutation pvalue dataframe
permdf.to_csv(savepathdir+file_path.split('/')[-1].split('.csv')[0]+'_%sC_%s_permu.csv.gz'%(
    T,D),compression='gzip')

In [26]:
## concat pheno df
phenodf = pd.concat(pheno_dfs,axis=1)

In [27]:
## Assign column names
phenodf.columns = [param+'_%sC_%s_p%s'%(T,D,i) for i in range(perms)]

In [28]:
## Save dataframe
phenodf.to_csv(savepathdir+file_path.split('/')[-1].split('.csv')[0]+'_%sC_%s_pheno.csv.gz'%(
    T,D),compression='gzip')